In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install multiprocess
!pip install tensorboardX
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 3.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path '/content/pyrouge/tools/ROUGE-1.5.5'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191620 sha256=e79aa0f2f6e60356c455faf91a7dd0058f15ec93817b870688ef9a9234580673
  Stored in directory: /root/.cache/pip/wheels/00/85/fd/ccd28e53c9f6a691e6ea96050a0cad95f9a4a6361269d765ca
Successfully built pyrouge
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 202.3 kB 2.6 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.h

In [4]:
import os
import json
from tqdm import tqdm
import re
import easydict
from rouge import Rouge

In [5]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid'


- 총 4만개는 학습시간이 60시간 걸림 그래서 각각 5%데이터로 해보자

In [6]:
def data_load(DATAPATH):
    filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
    filenames.sort()
    filenames
    list_dic = []

    for file in filenames:
      filelocation = os.path.join(DATAPATH, file)

      with open(filelocation, 'r') as json_file:
        data = json.load(json_file)['documents']
        data_len = round(len(data) * 0.05)
        data = data[:data_len]
        for x in tqdm (range(len(data))):
          text = data[x]['text']
          text = str(text).replace('"', "'")

          extractive = data[x]['extractive']
          for index, value in enumerate(extractive):
            if value == None:
              extractive[index] = 0

          p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
          texts = p.findall(text)

          sentences = []
          for t in texts:
            sentence = t[:-3]
            sentences.append(sentence)

          mydict = {}
          mydict['text'] = sentences
          mydict['extractive'] = extractive
          list_dic.append(mydict)

    return list_dic

In [7]:
list_dic = data_load(DATAPATH)

100%|██████████| 350/350 [00:00<00:00, 9149.99it/s]


In [8]:
len(list_dic)

2006

# transformer vs rnn vs classification
- colab computing 부족으로 실행중단
- 논문에서는 transformer가 가장 좋다고 소개했기 때문에 transformer를 가지고 최종 학습 및 평가 진행

In [9]:
selected_texts = []
for j in tqdm(range(len(list_dic))):
    selected_texts.append([list_dic[j]['text'][i] for i in list_dic[j]['extractive']])

input_text2 = []
for i in tqdm(range(len(list_dic))):
    input_text2.append(list_dic[i]['text'])

100%|██████████| 2006/2006 [00:00<00:00, 740714.31it/s]


In [10]:
import sys 
sys.path.append("/content/drive/MyDrive/인공지능/추출요약")
from SRC.train import new_inference

In [11]:
transformer_path = "/content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer/model_step_1000.pt"
rnn_path = "/content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_rnn/model_step_1000.pt"
classification_path = "/content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_classifier/model_step_1000.pt"

In [ ]:
transformer_hypotheses = []
for i in tqdm(range(len(input_text2))):
    transformer_hypotheses.append(new_inference(input_text2[i], transformer_path, "transformer", "0", "0",1))

rnn_hypotheses = []
for i in tqdm(range(len(input_text2))):
    rnn_hypotheses.append(new_inference(input_text2[i], rnn_path, "rnn", "0", "0",1))

classification_hypotheses = []
for i in tqdm(range(len(input_text2))):
    classification_hypotheses.append(new_inference(input_text2[i], classification_path, "classifier", "0", "0",1))

In [ ]:
transformer_references_list = []
transformer_hypotheses_list = []

for re, hy in tqdm(zip(selected_texts, transformer_hypotheses)):
    transformer_references_list.append(' '.join(re))
    transformer_hypotheses_list.append(' '.join(hy))

rnn_references_list = []
rnn_hypotheses_list = []

for re, hy in tqdm(zip(selected_texts, rnn_hypotheses)):
    rnn_references_list.append(' '.join(re))
    rnn_hypotheses_list.append(' '.join(hy))

classification_references_list = []
classification_hypotheses_list = []

for re, hy in tqdm(zip(selected_texts, classification_hypotheses)):
    classification_references_list.append(' '.join(re))
    classification_hypotheses_list.append(' '.join(hy))

In [ ]:
def rouge_f_r_summary(hypotheses, references):
    rouge = Rouge()

    scores = rouge.get_scores(hypotheses, references, avg=True)
    rouge_f_1 = scores['rouge-1']['f']
    rouge_r_1 = scores['rouge-1']['r']
    rouge_f_2 = scores['rouge-2']['f']
    rouge_r_2 = scores['rouge-2']['r']
    rouge_f_L = scores['rouge-l']['f']
    rouge_r_L = scores['rouge-l']['r']
    return rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L

rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L= rouge_f_r_summary(transformer_hypotheses_list, transformer_references_list)
print("--------------------------Rouge-F---------------------------")
print("Rouge-F_1:", rouge_f_1)
print("Rouge-F_2:", rouge_f_2)
print("Rouge-F_L:", rouge_f_L)
print("")
print("--------------------------Rouge-R---------------------------")
print("Rouge-R_1:", rouge_r_1)
print("Rouge-R_2:", rouge_r_2)
print("Rouge-R_L:", rouge_r_L)


In [ ]:
import pandas as pd

df = pd.DataFrame(zip([rouge_f_1], [rouge_f_2] , [rouge_f_L], [rouge_r_1], [rouge_r_2], [rouge_r_L] ),
                  columns=['rouge_f_1','rouge_f_L',  'rouge_f_2', 'rouge_r_1', 'rouge_r_2', 'rouge_r_L'])
df.to_csv("/content/drive/MyDrive/인공지능/추출요약/rouge_scores/transformer_1000_score.csv", mode='w', index = False)


In [ ]:
def rouge_f_r_summary(hypotheses, references):
    rouge = Rouge()

    scores = rouge.get_scores(hypotheses, references, avg=True)
    rouge_f_1 = scores['rouge-1']['f']
    rouge_r_1 = scores['rouge-1']['r']
    rouge_f_2 = scores['rouge-2']['f']
    rouge_r_2 = scores['rouge-2']['r']
    rouge_f_L = scores['rouge-l']['f']
    rouge_r_L = scores['rouge-l']['r']
    return rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L

rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L= rouge_f_r_summary(rnn_hypotheses_list, rnn_references_list)
print("--------------------------Rouge-F---------------------------")
print("Rouge-F_1:", rouge_f_1)
print("Rouge-F_2:", rouge_f_2)
print("Rouge-F_L:", rouge_f_L)
print("")
print("--------------------------Rouge-R---------------------------")
print("Rouge-R_1:", rouge_r_1)
print("Rouge-R_2:", rouge_r_2)
print("Rouge-R_L:", rouge_r_L)


In [ ]:
import pandas as pd

df = pd.DataFrame(zip([rouge_f_1], [rouge_f_2] , [rouge_f_L], [rouge_r_1], [rouge_r_2], [rouge_r_L] ),
                  columns=['rouge_f_1','rouge_f_L',  'rouge_f_2', 'rouge_r_1', 'rouge_r_2', 'rouge_r_L'])
df.to_csv("/content/drive/MyDrive/인공지능/추출요약/rouge_scores/rnn_1000_score.csv", mode='w', index = False)


In [ ]:
def rouge_f_r_summary(hypotheses, references):
    rouge = Rouge()

    scores = rouge.get_scores(hypotheses, references, avg=True)
    rouge_f_1 = scores['rouge-1']['f']
    rouge_r_1 = scores['rouge-1']['r']
    rouge_f_2 = scores['rouge-2']['f']
    rouge_r_2 = scores['rouge-2']['r']
    rouge_f_L = scores['rouge-l']['f']
    rouge_r_L = scores['rouge-l']['r']
    return rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L

rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L= rouge_f_r_summary(classification_hypotheses_list, classification_references_list)
print("--------------------------Rouge-F---------------------------")
print("Rouge-F_1:", rouge_f_1)
print("Rouge-F_2:", rouge_f_2)
print("Rouge-F_L:", rouge_f_L)
print("")
print("--------------------------Rouge-R---------------------------")
print("Rouge-R_1:", rouge_r_1)
print("Rouge-R_2:", rouge_r_2)
print("Rouge-R_L:", rouge_r_L)


In [ ]:
import pandas as pd

df = pd.DataFrame(zip([rouge_f_1], [rouge_f_2] , [rouge_f_L], [rouge_r_1], [rouge_r_2], [rouge_r_L] ),
                  columns=['rouge_f_1','rouge_f_L',  'rouge_f_2', 'rouge_r_1', 'rouge_r_2', 'rouge_r_L'])
df.to_csv("/content/drive/MyDrive/인공지능/추출요약/rouge_scores/classification_1000_score.csv", mode='w', index = False)


# 최종 모델 : transformer 5만 step

## 원본 본문 및 요약 추출

In [ ]:
selected_texts = []
for j in tqdm(range(len(list_dic))):
    selected_texts.append([list_dic[j]['text'][i] for i in list_dic[j]['extractive']])


100%|██████████| 2006/2006 [00:00<00:00, 263647.22it/s]


In [ ]:
selected_texts[0]

['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
 '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
 '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']

## 예측 추출요약 - 4만개 60시간 걸림 각각 5%데이터로로 해보자

In [ ]:
input_text2 = []
for i in tqdm(range(len(list_dic))):
    input_text2.append(list_dic[i]['text'])

100%|██████████| 2006/2006 [00:00<00:00, 963556.32it/s]


In [ ]:
input_text2[0]

['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
 '처분 등이 있은 날부터 1년을 경과하면',
 '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.',
 '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,',
 '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
 '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,',
 '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']

In [ ]:
len(input_text2)

2006

In [ ]:
import sys 
sys.path.append("/content/drive/MyDrive/인공지능/추출요약")
from SRC.train import new_inference

In [ ]:
new_inference

<function SRC.train.new_inference(input_data, test_from, encoder, visible_gpus, gpu_ranks, world_size)>

In [ ]:
test_from = "/content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result/model_step_50000.pt"

In [ ]:
# test = new_inference(input_text2[0], test_from, "transformer", "0", "0",1)

In [ ]:
# print(test)

In [ ]:
hypotheses = []
for i in tqdm(range(len(input_text2))):
    hypotheses.append(new_inference(input_text2[i], test_from, "transformer", "0", "0",1))

100%|██████████| 2006/2006 [3:07:54<00:00,  5.62s/it]


## 평가

### 1) 한 문장 평가

In [ ]:
# def rouge_f_r_summary(hypotheses, references):
#     rouge = Rouge()

#     reference = ' '.join(references)
#     hypothese = ' '.join(hypotheses)

#     scores = rouge.get_scores(hypothese, reference, avg=True)
#     rouge_f_1 = scores['rouge-1']['f']
#     rouge_r_1 = scores['rouge-1']['r']
#     rouge_f_2 = scores['rouge-2']['f']
#     rouge_r_2 = scores['rouge-2']['r']
#     rouge_f_L = scores['rouge-l']['f']
#     rouge_r_L = scores['rouge-l']['r']
#     return rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L

In [ ]:
# rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L= rouge_f_r_summary(test, selected_texts[0])
# print("--------------------------Rouge-F---------------------------")
# print("Rouge-F_1:", rouge_f_1)
# print("Rouge-F_2:", rouge_f_2)
# print("Rouge-F_L:", rouge_f_L)
# print("")
# print("--------------------------Rouge-R---------------------------")
# print("Rouge-R_1:", rouge_r_1)
# print("Rouge-R_2:", rouge_r_2)
# print("Rouge-R_L:", rouge_r_L)


### 전체 평가

In [ ]:
references_list = []
hypotheses_list = []

for re, hy in tqdm(zip(selected_texts, hypotheses)):
    references_list.append(' '.join(re))
    hypotheses_list.append(' '.join(hy))

2006it [00:00, 402572.91it/s]


In [ ]:
def rouge_f_r_summary(hypotheses, references):
    rouge = Rouge()

    scores = rouge.get_scores(hypotheses, references, avg=True)
    rouge_f_1 = scores['rouge-1']['f']
    rouge_r_1 = scores['rouge-1']['r']
    rouge_f_2 = scores['rouge-2']['f']
    rouge_r_2 = scores['rouge-2']['r']
    rouge_f_L = scores['rouge-l']['f']
    rouge_r_L = scores['rouge-l']['r']
    return rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L

rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L= rouge_f_r_summary(hypotheses_list, references_list)
print("--------------------------Rouge-F---------------------------")
print("Rouge-F_1:", rouge_f_1)
print("Rouge-F_2:", rouge_f_2)
print("Rouge-F_L:", rouge_f_L)
print("")
print("--------------------------Rouge-R---------------------------")
print("Rouge-R_1:", rouge_r_1)
print("Rouge-R_2:", rouge_r_2)
print("Rouge-R_L:", rouge_r_L)


--------------------------Rouge-F---------------------------
Rouge-F_1: 0.4774364113055629
Rouge-F_2: 0.4314373358893575
Rouge-F_L: 0.4745759762675957

--------------------------Rouge-R---------------------------
Rouge-R_1: 0.49247511414743644
Rouge-R_2: 0.4450575426622147
Rouge-R_L: 0.4895565404314038


In [ ]:
import pandas as pd

df = pd.DataFrame(zip([rouge_f_1], [rouge_f_2] , [rouge_f_L], [rouge_r_1], [rouge_r_2], [rouge_r_L] ),
                  columns=['rouge_f_1','rouge_f_L',  'rouge_f_2', 'rouge_r_1', 'rouge_r_2', 'rouge_r_L'])
df.to_csv("/content/drive/MyDrive/인공지능/추출요약/rouge_scores/transformer_50000_score.csv", mode='w', index = False)
